In [2]:
import os
import numpy as np
import pandas as pd
import random 
import time
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from glob import glob
%matplotlib inline

In [3]:
list_of_df = glob("line_of_sight_data_*")

In [4]:
list_of_df

['line_of_sight_data_1.csv',
 'line_of_sight_data_2.csv',
 'line_of_sight_data_3.csv',
 'line_of_sight_data_4.csv',
 'line_of_sight_data_5.csv']

In [5]:
list_df = [pd.read_csv(csvPath) for csvPath in list_of_df]

In [62]:
tdf = list_df[0]
tdf[:10]

,Unnamed: 0,Frame,Time_Stamp,BS1_coords,BS2_coords,Main_Obj_Coords,Obstacle1,Obstacle2,LOS_BS1,LOS_BS2
0,0,1,0.01,"(97, 61)","(202, 164)","(0, 140)","(-1, -1)","(80, 130)",1,1
1,1,2,0.02,"(97, 61)","(202, 164)","(0, 140)","(-1, -1)","(80, 130)",1,1
2,2,3,0.03,"(97, 61)","(202, 164)","(0, 140)","(-1, -1)","(80, 130)",1,1
3,3,4,0.04,"(97, 61)","(202, 164)","(0, 140)","(-1, -1)","(80, 130)",1,1
4,4,5,0.05,"(97, 61)","(202, 164)","(0, 140)","(-1, -1)","(80, 130)",1,1
5,5,6,0.06,"(97, 61)","(202, 164)","(0, 140)","(-1, -1)","(80, 130)",1,1
6,6,7,0.07,"(97, 61)","(202, 164)","(1, 140)","(-1, -1)","(80, 130)",1,1
7,7,8,0.08,"(97, 61)","(202, 164)","(1, 140)","(-1, -1)","(80, 130)",1,1
8,8,9,0.09,"(97, 61)","(202, 164)","(1, 140)","(-1, -1)","(80, 130)",1,1
9,9,10,0.10,"(97, 61)","(202, 164)","(1, 140)","(-1, -1)","(80, 130)",1,1


In [31]:
tdf=tdf.iloc[:1530]

In [32]:
req_cols = ["Time_Stamp", "Main_Obj_Coords", "BS1_coords", "Obstacle1", "Obstacle2", "LOS_BS1"]

In [33]:
tdf = tdf[req_cols]

In [34]:
for i,row in enumerate(tdf):
    print(i,":",row)

0 : Time_Stamp
1 : Main_Obj_Coords
2 : BS1_coords
3 : Obstacle1
4 : Obstacle2
5 : LOS_BS1


In [53]:
tdf.iloc[0]["LOS_BS1"]

1

In [48]:
(tdf.iloc[0]["Main_Obj_Coords"])[1:-1]

'0, 140'

In [49]:
x,y=tuple(map(int, '-1, -1'.split(',')))

In [51]:
x

-1

In [54]:
list((9,5))

[9, 5]

In [28]:
def euclidean_dist(point1, point2):
    p1 = np.array(point1)
    p2 = np.array(point2)
    dist = np.linalg.norm(p1 - p2)
    return dist

In [55]:
def create_data(list_df, req_cols):
    data_list1 = []
    data_list2 = []
    for df in list_df:
        data1 = []
        data2 = []
        df = df[req_cols]
        df = df.iloc[:len(df)-6]
        for i in range(len(df)):
            temp1 = []
            temp2 = []
            time = df.iloc[i]["Time_Stamp"]
            main_x, main_y = tuple(map(int, ((df.iloc[i]["Main_Obj_Coords"])[1:-1]).split(',')))
            bs_coords = tuple(map(int, ((df.iloc[i]["BS1_coords"])[1:-1]).split(',')))
            obs1_coords = tuple(map(int, ((df.iloc[i]["Obstacle1"])[1:-1]).split(',')))
            obs2_coords = tuple(map(int, ((df.iloc[i]["Obstacle2"])[1:-1]).split(',')))
            dist_from_bs = euclidean_dist((main_x, main_y), bs_coords)
            dist_from_obs1 = euclidean_dist((main_x, main_y), obs1_coords)
            dist_from_obs2 = euclidean_dist((main_x, main_y), obs2_coords)
            los_bs1 = df.iloc[i]["LOS_BS1"]
            temp1.append(time)
            temp2.append(time)
            temp1.append(main_x)
            temp1.append(main_y)
            temp2.append(main_x)
            temp2.append(main_y)
            temp1.append(dist_from_bs)
            temp1.append(dist_from_obs1)
            temp1.append(dist_from_obs2)
            temp2 += list(bs_coords)
            temp2 += list(obs1_coords)
            temp2 += list(obs2_coords)
            temp1.append(los_bs1)
            temp2.append(los_bs1)
            data1.append(temp1)
            data2.append(temp2)
        data_list1.append(data1)
        data_list2.append(data2)
    return data_list1, data_list2

In [56]:
data_list1, data_list2 = create_data(list_df, req_cols)

In [57]:
data_list1 = np.array(data_list1)
data_list2 = np.array(data_list2)

In [65]:
data_list1[0]

array([[1.00000000e-02, 0.00000000e+00, 1.40000000e+02, ...,
        1.41003546e+02, 8.06225775e+01, 1.00000000e+00],
       [2.00000000e-02, 0.00000000e+00, 1.40000000e+02, ...,
        1.41003546e+02, 8.06225775e+01, 1.00000000e+00],
       [3.00000000e-02, 0.00000000e+00, 1.40000000e+02, ...,
        1.41003546e+02, 8.06225775e+01, 1.00000000e+00],
       ...,
       [1.52800000e+01, 2.54000000e+02, 1.40000000e+02, ...,
        2.91386341e+02, 4.80520551e+01, 0.00000000e+00],
       [1.52900000e+01, 2.54000000e+02, 1.40000000e+02, ...,
        2.91386341e+02, 4.80520551e+01, 0.00000000e+00],
       [1.53000000e+01, 2.54000000e+02, 1.40000000e+02, ...,
        2.91386341e+02, 4.80520551e+01, 0.00000000e+00]])

In [69]:
datax = [i.tolist()[:-1] for i in data_list1[0]]
datay = [i.tolist()[-1] for i in data_list1[0]]

In [74]:
datax[:9]

[[0.01, 0.0, 140.0, 125.09996003196804, 141.00354605470034, 80.62257748298549],
 [0.02, 0.0, 140.0, 125.09996003196804, 141.00354605470034, 80.62257748298549],
 [0.03, 0.0, 140.0, 125.09996003196804, 141.00354605470034, 80.62257748298549],
 [0.04, 0.0, 140.0, 125.09996003196804, 141.00354605470034, 80.62257748298549],
 [0.05, 0.0, 140.0, 125.09996003196804, 141.00354605470034, 80.62257748298549],
 [0.06, 0.0, 140.0, 125.09996003196804, 141.00354605470034, 80.62257748298549],
 [0.07, 1.0, 140.0, 124.32618388738553, 141.01418368376991, 79.63039620647382],
 [0.08, 1.0, 140.0, 124.32618388738553, 141.01418368376991, 79.63039620647382],
 [0.09, 1.0, 140.0, 124.32618388738553, 141.01418368376991, 79.63039620647382]]

In [66]:
timestep = 9

In [77]:
def createTrainDatasetFromData(timestep, data):
    datax = [i.tolist()[:-1] for i in data]
    datay = [i.tolist()[-1] for i in data]
    X = []
    y = []
    for i in range(len(data)-timestep):
        X.append(datax[i:i+timestep])
        y.append(datay[i+timestep])
    return X,y

In [84]:
def createTrainDataset(timestep, data_list):
    X = []
    y = []
    for data in data_list:
        tx,ty = createTrainDatasetFromData(timestep, data)
        X += tx
        y += ty
    return X, y

In [114]:
timestep = 100

In [115]:
X1, y1 = createTrainDataset(timestep, data_list1)
X2, y2 = createTrainDataset(timestep, data_list2)

In [116]:
fX1, fy1 = np.array(X1), np.array(y1)
fX2, fy2 = np.array(X2), np.array(y2)

In [122]:
def LSTM_Model(inp_shape):
    
    model = Sequential()
    
    model.add(LSTM(units=64, input_shape=inp_shape, activation='relu', return_sequences=True))
#     model.add(Dropout(0.2))
    
    model.add(LSTM(units=64, activation='relu', return_sequences=True))
#     model.add(Dropout(0.2))
    
    model.add(LSTM(units=64, activation='relu'))
#     model.add(Dropout(0.2))
    
#     model.add(CuDNNLSTM(units=64, input_shape=inp_shape, return_sequences=True))
#     model.add(Dropout(0.2))
    
#     model.add(CuDNNLSTM(units=64, return_sequences=True))
#     model.add(Dropout(0.2))
    
#     model.add(CuDNNLSTM(units=64))
#     model.add(Dropout(0.2))
    
    model.add(Dense(units=128, activation='relu'))
#     model.add(Dropout(0.2))
    
    model.add(Dense(units=64, activation='relu'))
#     model.add(Dropout(0.2))
    
    model.add(Dense(units=1, activation='softmax'))
        
    model.compile(loss='binary_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])
    
    return model

In [123]:
inp_shape = fX2.shape[1:]
inp_shape

(100, 9)

In [124]:
lstm_model = LSTM_Model(inp_shape)

In [125]:
lstm_model.fit(fX2, fy2, epochs=10, batch_size=32)

Epoch 1/10
224/224 [==============================] - 166s 727ms/step - loss: nan - accuracy: 0.6531
Epoch 2/10
224/224 [==============================] - 164s 734ms/step - loss: nan - accuracy: 0.6559
Epoch 3/10
224/224 [==============================] - 177s 789ms/step - loss: nan - accuracy: 0.6559
Epoch 4/10
224/224 [==============================] - 167s 747ms/step - loss: nan - accuracy: 0.6559
Epoch 5/10
224/224 [==============================] - 158s 705ms/step - loss: nan - accuracy: 0.6559
Epoch 6/10
224/224 [==============================] - 167s 744ms/step - loss: nan - accuracy: 0.6559
Epoch 7/10
224/224 [==============================] - 180s 806ms/step - loss: nan - accuracy: 0.6559
Epoch 8/10
224/224 [==============================] - 175s 783ms/step - loss: nan - accuracy: 0.6559
Epoch 9/10
224/224 [==============================] - 173s 772ms/step - loss: nan - accuracy: 0.6559
Epoch 10/10
224/224 [==============================] - 176s 784ms/step - loss: nan - accura